In [1]:
from bs4 import BeautifulSoup
import os,io
import parser,urllib.request,requests
from lxml import html,etree
import re
import pandas as pd

import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
opener = urllib.request.FancyURLopener({})
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
f = pd.read_html(url,header=0)
df = pd.DataFrame(f[0])
df.head()

C:\Users\Daniel\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

rename the column to format

In [4]:
df.rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'},inplace=True)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True,inplace=True)

In [6]:
df.shape

(211, 3)

These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

The neighborhood will be combined in accordance to postalcode and borough (grouped).

In [8]:
df = df.groupby(['PostalCode','Borough']).Neighborhood.agg([('Neighborhood',', '.join)])
df.reset_index(inplace=True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

This basically pinpoint where neighborhood content is 'Not assigned' and copy/paste its borough content into it.

In [10]:
df.loc[df[df['Neighborhood']=='Not assigned']['Neighborhood'].index.values[0]]['Neighborhood'] = df.loc[df[df['Neighborhood']=='Not assigned']['Neighborhood'].index.values[0]]['Borough']

In [11]:
df.shape

(103, 3)